# Dots and Boxes in Python
Dots and Boxes is a simple Strategy game developed in the 1880s in France.
This program will attempt to recreate it in Python and apply a Search Tree algorithm to the results.

## Dots and Boxes logic
The game Dots and Boxes is a territory conquering game, focusing on capturing boxes that are not owned by any entity. A single box is considered owned when the playing character places the final line around it. This increments that players score, and the highest score wins.

The copy_game_state method copies the state outside of the algorithm using the games variables and preserves the current game state. Deep copy is used because the default shallow copy would modify the board.

In [69]:
import copy

In [70]:
class DotsAndBoxes:
    def __init__(self, rows=3, cols=3):
        self.rows = rows
        self.cols = cols
        self.h_edges = [[False for _ in range(cols)] for _ in range(rows + 1)]
        self.v_edges = [[False for _ in range(cols + 1)] for _ in range(rows)]
        self.boxes = [[0 for _ in range(cols)] for _ in range(rows)]
        self.current_player = 1
        self.scores = {1: 0, 2: 0}

    def display_board(self):
        print("Current Board:")

        for row in range(self.rows + 1):
            line = ""
            for col in range(self.cols):
                line += "*"
                if self.h_edges[row][col]:
                    line += "———"
                else:
                    line += "   "
            line += "*"
            print(line)

            if row < self.rows:
                line = ""
                for col in range(self.cols + 1):
                    if self.v_edges[row][col]:
                        line += "|"
                    else:
                        line += " "

                    if col < self.cols:
                        box_val = self.boxes[row][col]
                        if box_val == 0:
                            line += "   "
                        else:
                            line += f" {box_val} "
                print(line)
        print(f"Scores - Player: {self.scores[1]}, Computer: {self.scores[2]}")
        print(f"Player: {self.current_player}")

    def get_possible_moves(self): # Also needed for copying with SearchTreeNode
        moves = []

        for row in range(self.rows + 1):
            for col in range(self.cols):
                if not self.h_edges[row][col]:
                    moves.append(('h', row, col))

        for row in range(self.rows):
            for col in range(self.cols + 1):
                if not self.v_edges[row][col]:
                    moves.append(('v', row, col))

        return moves

    def is_game_over(self):
            return len(self.get_possible_moves()) == 0

    def get_winner(self):
            if self.scores[1] > self.scores[2]:
                return 1
            elif self.scores[2] > self.scores[1]:
                return 2
            else:
                return 0

    def list_moves(self):
            moves = self.get_possible_moves()
            print(f"\nAvailable moves for Player {self.current_player}:")
            for i, move in enumerate(moves):
                edge_type, row, col = move
                if edge_type == 'h':
                    print(f"{i}: Horizontal edge at row {row}, col {col}")
                else:
                    print(f"{i}: Vertical edge at row {row}, col {col}")
            return moves

    def get_move(self):
        moves = self.list_moves()
        while True:
            try:
                choice = input(f"Player {self.current_player}, enter move number (0-{len(moves)-1}): ")
                choice = int(choice)

                if 0 <= choice < len(moves):
                    return moves[choice]
                else:
                    print(f"Please enter a number between 0 and {len(moves)-1}")

            except ValueError:
                print("Please enter a valid number")
                return None

    def is_box_complete(self, box_row, box_col):
        if box_row < 0 or box_row >= self.rows or box_col < 0 or box_col >= self.cols:
            return False

        top = self.h_edges[box_row][box_col]
        bottom = self.h_edges[box_row + 1][box_col]
        left = self.v_edges[box_row][box_col]
        right = self.v_edges[box_row][box_col + 1]

        return top and bottom and left and right

    def make_move(self, move):
        edge_type, row, col = move

        if edge_type == 'h':
            if self.h_edges[row][col]:
                return False  # Edge already drawn
            self.h_edges[row][col] = True
        else:  # vertical edge
            if self.v_edges[row][col]:
                return False  # Edge already drawn
            self.v_edges[row][col] = True

        boxes_completed = 0
        boxes_to_check = []

        if edge_type == 'h': # Horizontal edge can complete boxes above and below
            if row > 0:
                boxes_to_check.append((row - 1, col))
            if row < self.rows:
                boxes_to_check.append((row, col))
        else:
            if col > 0:
                boxes_to_check.append((row, col - 1))
            if col < self.cols:
                boxes_to_check.append((row, col))

        for box_row, box_col in boxes_to_check:
            if (self.boxes[box_row][box_col] == 0 and
                self.is_box_complete(box_row, box_col)):

                self.boxes[box_row][box_col] = self.current_player
                self.scores[self.current_player] += 1
                boxes_completed += 1

        if boxes_completed == 0: # Switch player only if no boxes were completed
            self.current_player = 2 if self.current_player == 1 else 1

        return boxes_completed

    def get_opponent(self, player=None):
        return 2 if player == 1 else 1

    def evaluate_position(self):
        my_score = self.scores[self.current_player]
        opponent_score = self.scores[self.get_opponent()]
        return my_score - opponent_score

## Search Tree Algorithm
The search tree algorithm, otherwise known as the Minimax Tree algorithm, is a recursive algorithm that uses the end state of the game to decide on a move to make. The algorithm is guided by a Hueristic function, deciding which move is best.

## Search Tree Node Class
The SearchTreeNode class below is responsible for creating and manipulating the Search Tree itself. The Search Tree Algorithm here, otherwise known as "Minimax" Tree algorithm, recursively draws all possible combinations of the board. It assigns a value based on the maximum or minimum value available at the time.

<hr>

The problem with using this algorithm is the player changing mechanism of the game itself. The game here adds complexity to the algorithm, as the game continues if a box is completed by the current player. This lead to the algorithm having to be recalled if a move completes a box, hence the  `if boxes_completed` condition within both the algorithm and the heuristic method. This method call ensures that the maximizing or minimizing value is kept consistent. For this reason, it was decided that the algorithm given as a sample would have to be rewritten, as it only handled alternating player inputs, where Dots and Boxes does not. It could very well be the case here that MAX gets two or three depths of the tree in a row.

<hr>

### get_best_move() Hueristic
The Hueristic function here returns the move that resulted in the highest available score. Or the first move that has the highest score, as this can be multiple moves. The recursive depth here is set to 3, for performance optimisation.

In [71]:
class SearchTreeNode:
    def __init__(self, board_instance): # Initializer had to be inserted to control the games state in this cell.
        self.game = board_instance

    def SearchTreeAlgorithm(self, depth, maximizing_player, minimizing_player): # Bool for Maximizing player.
        if self.game.is_game_over() or depth == 0: # Base Case
            my_score = self.game.scores[minimizing_player] # Record both parties scores for min and max
            opponent_score = self.game.scores[self.game.get_opponent(minimizing_player)]
            return my_score - opponent_score

        if maximizing_player:
            max_eval = float('-inf')
            for move in self.game.get_possible_moves():
                game_copy = copy.deepcopy(self.game)
                boxes_completed = game_copy.make_move(move)

                new_node = SearchTreeNode(game_copy) # Create a new SearchTreeNode for the copied game state
                if boxes_completed > 0:
                    eval_score = new_node.SearchTreeAlgorithm(depth - 1, True, minimizing_player)
                else:
                    eval_score = new_node.SearchTreeAlgorithm(depth - 1, False, minimizing_player)

                max_eval = max(max_eval, eval_score)
            return max_eval
        else: # Minimizing player
            min_eval = float('inf')
            for move in self.game.get_possible_moves():
                game_copy = copy.deepcopy(self.game)
                boxes_completed = game_copy.make_move(move)

                new_node = SearchTreeNode(game_copy)
                if boxes_completed > 0: # Needs to be recalled if a box is completed.
                    eval_score = new_node.SearchTreeAlgorithm(depth - 1, False, minimizing_player)
                else:
                    eval_score = new_node.SearchTreeAlgorithm(depth - 1, True, minimizing_player)
                min_eval = min(min_eval, eval_score)
            return min_eval

    #Heuristic method to assess the values in each depth level of the tree. depth=3
    def get_best_move(self, depth=3): # Search for the best move using Copies to SIMULATE what would happen rather than modifying the actual game board.
        if not self.game.get_possible_moves():
            return None

        best_move = None
        best_score = float('-inf') # Initialize to minimum value

        print(f"Computer considering {len(self.game.get_possible_moves())} possible moves at depth {depth}")

        for i, move in enumerate(self.game.get_possible_moves()):
            game_copy = copy.deepcopy(self.game)
            boxes_completed = game_copy.make_move(move)

            new_node = SearchTreeNode(game_copy) # Create a new SearchTreeNode for the copied game state

            if boxes_completed > 0: # Needs to be recalled if a box is completed.
                score = new_node.SearchTreeAlgorithm(depth - 1, True, self.game.current_player)
            else:
                score = new_node.SearchTreeAlgorithm(depth - 1, False, self.game.current_player)

            print(f"Move {i}: {move} -> Score: {score}")

            if score > best_score:
                best_score = score
                best_move = move

        print(f"Best move: {best_move} with score: {best_score}")
        return best_move # Returns move with highest possible score.

## Play game
This method brings the previous cells together, and allows the programmer to set the AI depth of the program.

In [72]:
def play_game(player=1, ai_depth=3):
    game = DotsAndBoxes()
    ai = SearchTreeNode(game)

    print(f"Welcome to Dots and Boxes vs Computer!")
    print(f"AI search depth: {ai_depth}")

    game.display_board()

    while not game.is_game_over():
        if game.current_player == player:
            move = game.get_move()
            if move is None:
                break
        else:
            move = ai.get_best_move(ai_depth)
            print(f"AI plays: {move}")

        boxes_completed = game.make_move(move)

        if boxes_completed > 0:
            player_name = "You" if game.current_player == player else "Computer"
            print(f"{player_name} completed {boxes_completed} box(es) and gets another turn!")

        game.display_board()

    if game.is_game_over():
        winner = game.get_winner()
        if winner == 0:
            print("It's a tie!")
        elif winner == player:
            print("Congratulations! You won!")
        else:
            print("Computer wins! Oh well.")

    return winner

play_game(player=1, ai_depth=3)

Welcome to Dots and Boxes vs Computer!
AI search depth: 3
Current Board:
*   *   *   *
             
*   *   *   *
             
*   *   *   *
             
*   *   *   *
Scores - Player: 0, Computer: 0
Player: 1

Available moves for Player 1:
0: Horizontal edge at row 0, col 0
1: Horizontal edge at row 0, col 1
2: Horizontal edge at row 0, col 2
3: Horizontal edge at row 1, col 0
4: Horizontal edge at row 1, col 1
5: Horizontal edge at row 1, col 2
6: Horizontal edge at row 2, col 0
7: Horizontal edge at row 2, col 1
8: Horizontal edge at row 2, col 2
9: Horizontal edge at row 3, col 0
10: Horizontal edge at row 3, col 1
11: Horizontal edge at row 3, col 2
12: Vertical edge at row 0, col 0
13: Vertical edge at row 0, col 1
14: Vertical edge at row 0, col 2
15: Vertical edge at row 0, col 3
16: Vertical edge at row 1, col 0
17: Vertical edge at row 1, col 1
18: Vertical edge at row 1, col 2
19: Vertical edge at row 1, col 3
20: Vertical edge at row 2, col 0
21: Vertical edge at row 2, 

2

### Testing
Testing that all moves are considered.

In [73]:
cb = DotsAndBoxes(3,3)
cb.display_board()
cb.get_possible_moves()
test_node = SearchTreeNode(cb)
test_node.get_best_move()

Current Board:
*   *   *   *
             
*   *   *   *
             
*   *   *   *
             
*   *   *   *
Scores - Player: 0, Computer: 0
Player: 1
Computer considering 24 possible moves at depth 3
Move 0: ('h', 0, 0) -> Score: 0
Move 1: ('h', 0, 1) -> Score: 0
Move 2: ('h', 0, 2) -> Score: 0
Move 3: ('h', 1, 0) -> Score: 0
Move 4: ('h', 1, 1) -> Score: 0
Move 5: ('h', 1, 2) -> Score: 0
Move 6: ('h', 2, 0) -> Score: 0
Move 7: ('h', 2, 1) -> Score: 0
Move 8: ('h', 2, 2) -> Score: 0
Move 9: ('h', 3, 0) -> Score: 0
Move 10: ('h', 3, 1) -> Score: 0
Move 11: ('h', 3, 2) -> Score: 0
Move 12: ('v', 0, 0) -> Score: 0
Move 13: ('v', 0, 1) -> Score: 0
Move 14: ('v', 0, 2) -> Score: 0
Move 15: ('v', 0, 3) -> Score: 0
Move 16: ('v', 1, 0) -> Score: 0
Move 17: ('v', 1, 1) -> Score: 0
Move 18: ('v', 1, 2) -> Score: 0
Move 19: ('v', 1, 3) -> Score: 0
Move 20: ('v', 2, 0) -> Score: 0
Move 21: ('v', 2, 1) -> Score: 0
Move 22: ('v', 2, 2) -> Score: 0
Move 23: ('v', 2, 3) -> Score: 0
Best move: ('

('h', 0, 0)

## Testing an Ongoing Game

In [79]:
mid_game = DotsAndBoxes(3, 3) # Create a new game board
mid_game.h_edges[0][0] = True # Manually add edges
mid_game.h_edges[0][1] = True
mid_game.h_edges[1][0] = True
mid_game.h_edges[2][0] = True
mid_game.v_edges[2][0] = True #
mid_game.v_edges[1][0] = True
mid_game.v_edges[0][1] = True
mid_game.v_edges[1][1] = True
mid_game.v_edges[0][1] = True
mid_game.scores[1] += 1
mid_game.current_player = 2 # Computer move
mid_game.display_board()
node = SearchTreeNode(mid_game)
best_move = node.get_best_move(depth=3)
print(f"AI's best move from this mid-game state: {best_move}")

Current Board:
*———*———*   *
    |        
*———*   *   *
|   |        
*———*   *   *
|            
*   *   *   *
Scores - Player: 1, Computer: 0
Player: 2
Computer considering 16 possible moves at depth 3
Move 0: ('h', 0, 2) -> Score: -2
Move 1: ('h', 1, 1) -> Score: -3
Move 2: ('h', 1, 2) -> Score: -2
Move 3: ('h', 2, 1) -> Score: -2
Move 4: ('h', 2, 2) -> Score: -2
Move 5: ('h', 3, 0) -> Score: -3
Move 6: ('h', 3, 1) -> Score: -2
Move 7: ('h', 3, 2) -> Score: -2
Move 8: ('v', 0, 0) -> Score: 0
Move 9: ('v', 0, 2) -> Score: -3
Move 10: ('v', 0, 3) -> Score: -2
Move 11: ('v', 1, 2) -> Score: -2
Move 12: ('v', 1, 3) -> Score: -2
Move 13: ('v', 2, 1) -> Score: -3
Move 14: ('v', 2, 2) -> Score: -2
Move 15: ('v', 2, 3) -> Score: -2
Best move: ('v', 0, 0) with score: 0
AI's best move from this mid-game state: ('v', 0, 0)


## Results and Conclusions
In the cell above, the algorithm is called on a board that has a move that would complete a box. The move is v,0,0 which is successfully chosen when called.

The algorithm always chose a good move, and this always resulted in the AI winning over the player in a 3x3 game. In testing, I was not able to beat it. Following the perfect strategy, only a draw could be acheived against it.